In [768]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np

from PIL import Image

import pickle as pkl

import matplotlib.pyplot as plt
%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data parser

In [1466]:
class BatchLoader():
    def __init__(self, features, labels):
        self.features = features
        self.reference_features = self.random_pairs(features, labels)
        self.labels = labels
        
    def random_pairs(self, X, labels):
        Y = X.copy()
        for l in range(labels.shape[1]):
            inds = np.where(labels[:, l])[0]
            inds_pairs = np.random.permutation(inds)
            Y[inds, :] = X[inds_pairs, :]
        return Y
    
    def batch_load(self, start, end):
        if start == 0:
            idx = np.r_[:self.features.shape[0]]
            np.random.shuffle(idx)
            self.features = self.features[idx]
            self.reference_features = self.reference_features[idx]
            self.labels = self.labels[idx]
            
        if end > self.features.shape[0]:
            end = self.features.shape[0]
            
        return self.features[start:end], self.reference_features[start:end], self.labels[start:end]

In [1467]:
# features_train = np.load('Data/DEdata/features_train.npy').astype('float32')
# labels_train = np.load('Data/DEdata/labels_train.npy').astype('float32')
loader = BatchLoader(features_train, labels_train)

In [4]:
features_train = np.zeros((80, 600, 2048), dtype=np.float32)
for i in range(80):
    with open('Data/PickledClasses/' + str(i), 'rb') as f:
        data = pkl.load(f)
    features_train[i] = data

In [5]:
np.save('Data/DEdata/features_train.npy', features_train.reshape(-1, 2048))

# Meta-learning models

In [1359]:
class DeltaEncoder(nn.Module):
    def __init__(self, input_size=2048, hidden_size=8192, neck_size=16):
        encoder = nn.Sequential(
            nn.Linear(input_size * 2, hidden_size),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.5),
            
            nn.Linear(hidden_size, neck_size),
        )
        
        decoder = nn.Sequential(
            nn.Linear(input_size + neck_size, hidden_size),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.5),
            
            nn.Linear(hidden_size, input_size),
        )
        dropout = nn.Dropout(0.5)
        
        super(DeltaEncoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.dropout = dropout

    def forward(self, X1, X2):
        out = self.dropout(X1)
        out = torch.cat((out, X2), dim=1)
        out = self.encoder(out)
        
        out = torch.cat((X2, out), dim=1)
        out = self.decoder(out)
        return out

In [1406]:
G = DeltaEncoder(2048, 512, 8).to(device)

# Meta-learning phase

In [1407]:
MAE = nn.L1Loss(reduction='none')
MSE = nn.MSELoss(reduction='none')
def weighted_MAE(predict, target):
    batch_size = predict.shape[0]
    feature_size = predict.shape[1]

    substract_norm = MSE(predict, target)
    L2_norms = torch.sum(substract_norm, dim=1) + 10e-7
    weights = substract_norm / L2_norms.reshape((batch_size, 1)).expand((batch_size, feature_size))

    substract = MAE(predict, target)
    losses = torch.sum(substract * weights, dim=1)
    loss = torch.mean(losses)
    return loss

In [1408]:
# optimizer = torch.optim.Adam(G.parameters(), lr=10e-5)
optimizer = torch.optim.Adam(G.parameters(), lr=10e-5)

In [1418]:
batch_size = 512
train_size = 48000

for epoch in range(2):
    for i in range(train_size // batch_size):
        features, reference_features, labels = loader.batch_load(i * batch_size, (i + 1) * batch_size)
        features = torch.tensor(features, device=device, dtype=torch.float32, requires_grad=False)
        reference_features = torch.tensor(reference_features, device=device, dtype=torch.float32, requires_grad=False)
        predict = G(features, reference_features)
        
        loss = weighted_MAE(predict, features)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i % 10 == 0):
            print('Epoch {} Loss {}'.format(epoch, loss.detach()))

Epoch 0 Loss 3.8619093894958496
Epoch 0 Loss 3.8883285522460938
Epoch 0 Loss 3.8175785541534424
Epoch 0 Loss 3.8877413272857666
Epoch 0 Loss 3.918534278869629
Epoch 0 Loss 3.9437286853790283
Epoch 0 Loss 3.9527225494384766
Epoch 0 Loss 3.923792600631714
Epoch 0 Loss 3.873991012573242
Epoch 0 Loss 3.881967544555664
Epoch 1 Loss 3.905362606048584
Epoch 1 Loss 3.903092861175537
Epoch 1 Loss 3.920227289199829
Epoch 1 Loss 3.8377187252044678
Epoch 1 Loss 3.878110885620117
Epoch 1 Loss 3.9147236347198486
Epoch 1 Loss 3.88032603263855
Epoch 1 Loss 3.8578286170959473
Epoch 1 Loss 3.87811541557312
Epoch 1 Loss 3.854017734527588


# Generation & storing new samples

In [1419]:
class DeltaEncoderGenerator(nn.Module):
    def __init__(self, input_size=2048, hidden_size=8192, neck_size=16):
        encoder = nn.Sequential(
            nn.Linear(input_size * 2, hidden_size),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.5),
            
            nn.Linear(hidden_size, neck_size),
        )
        
        decoder = nn.Sequential(
            nn.Linear(input_size + neck_size, hidden_size),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.5),
            
            nn.Linear(hidden_size, input_size),
        )
        dropout = nn.Dropout(0.5)
        
        super(DeltaEncoderGenerator, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.dropout = dropout

    def forward(self, X1, X2, shot):
        out = self.dropout(X1)
        out = torch.cat((out, X2), dim=1)
        out = self.encoder(out)
        
        out = torch.cat((shot, out), dim=1)
        out = self.decoder(out)
        return out

In [1421]:
G_trained = DeltaEncoderGenerator(2048, 512, 8).to(device)
G_trained.load_state_dict(G.state_dict())

<All keys matched successfully>

In [1468]:
num_shots = 1
episode = torch.zeros(1, 5, num_shots, 2048, device=device, requires_grad=False)

for i in range(5):
    with open('Data/PickledClasses/' + str(95 + i), 'rb') as f:
        data = pkl.load(f)
        
    shot_numbers = np.random.randint(0, 600, size=num_shots)
    episode[0][i][:num_shots] = data[shot_numbers]
    
episode.shape

torch.Size([1, 5, 1, 2048])

In [1469]:
# store samples to cpu!

batch_size = 128
gen_size = 1024
train_size = 48000
class_data = torch.zeros(gen_size, 2048, device=device, dtype=torch.float32, requires_grad=False)

for class_num in range(5):
    indices = np.random.randint(low=0, high=train_size // batch_size, size=gen_size // batch_size)
    j = 0
    for i in indices:
        features, reference_features, labels = loader.batch_load(i * batch_size, (i + 1) * batch_size)
        features = torch.tensor(features, device=device, dtype=torch.float32, requires_grad=False)
        reference_features = torch.tensor(reference_features, device=device, dtype=torch.float32, requires_grad=False)
        
        shot = episode[0][class_num].expand(batch_size, 2048)
        class_data[j * batch_size:(j + 1) * batch_size] = G_trained(features, reference_features, shot).detach()
        j += 1
        
    with open('Data/DEFeatures/' + str(class_num), 'wb') as f:
        pkl.dump(class_data, f)
    print('Class {} processed.'.format(class_num))

Class 0 processed.
Class 1 processed.
Class 2 processed.
Class 3 processed.
Class 4 processed.


In [1323]:
batch_size = 128
class_size = 1024
train_size = 48000
pack_features = np.zeros((5, 2, 1024, 2048), dtype=np.float32)

total_indices = np.random.permutation(train_size // batch_size)
for class_num in range(5):
    indices = total_indices[class_num * (class_size // batch_size):(class_num + 1) * (class_size // batch_size)]
    j = 0
    for i in indices:
        features, reference_features, labels = loader.batch_load(i * batch_size, (i + 1) * batch_size)
        pack_features[class_num][0][j * batch_size:(j + 1) * batch_size] = features
        pack_features[class_num][1][j * batch_size:(j + 1) * batch_size] = reference_features
        j += 1
        
with open('Data/SynthMaterial/0', 'wb') as f:
    pkl.dump({'features': pack_features}, f)
print('Package processed.')

Package processed.


In [14]:
pack_features.shape

(5, 2, 1024, 2048)

# Training target classyfier

In [1470]:
class BatchLoader():
    def __init__(self, class_size, num_classes, first_class, batch_size, batches_in_buff, path):
        self.class_size = class_size
        self.num_classes = num_classes
        self.first_class = first_class
        self.batch_size = batch_size
        self.batches_in_buff = batches_in_buff
        self.path = path
        
        self.indices = np.random.permutation(num_classes * class_size)
        self.buff_size = batches_in_buff * batch_size
        self.buff = [{'label': 0, 'features': torch.zeros(2048, device=device)} for i in range(self.buff_size)]
        self.buff_num = 0
    
    def buff_gen(self, buff_num):
        buff_indices = self.indices[buff_num * self.buff_size:(buff_num + 1) * self.buff_size]

        for i in range(self.num_classes):
            with open(self.path + str(self.first_class + i), 'rb') as f:
                class_data = pkl.load(f)

            class_indices = np.where(((buff_indices < (i + 1) * self.class_size) & (buff_indices >= i * self.class_size)))[0]
            for j in class_indices:
                self.buff[j] = {
                    'label': i,
                    'features': class_data[buff_indices[j] % self.class_size]
                }
    
    def batch_load(self, i):
        buff_i = i % self.batches_in_buff
        if (buff_i == 0):
            self.buff_gen(self.buff_num)
            self.buff_num += 1
            
        return self.buff[buff_i * self.batch_size:(buff_i + 1) * self.batch_size]

In [1471]:
class Classyfier(nn.Module):
    def __init__(self):
        fc_layers = nn.Sequential(
            nn.Linear(2048, 5),
            # nn.Linear(512, 256),
            # nn.Linear(256, 5),
            nn.Softmax(dim=1)
        )
        super(Classyfier, self).__init__()
        self.fc = fc_layers
        
    def forward(self, x):
        out = self.fc(x)
        return out

In [1472]:
classyfier = Classyfier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classyfier.parameters(), lr=0.0001)

In [1473]:
class_size = 1024 # + 21997
num_classes = 5
first_class = 0
train_size = class_size * num_classes
batch_size = 128
batches_in_buff = 128
buff_size = batch_size * batches_in_buff

for epoch in range(3):
    loader = BatchLoader(class_size, num_classes, first_class, batch_size, batches_in_buff, 'Data/DEFeatures/')
    for i in range(train_size // batch_size):
        batch_tuple = loader.batch_load(i)
        images = torch.zeros(batch_size, 2048, device=device, requires_grad=False)
        labels = torch.zeros(batch_size, device=device, requires_grad=False, dtype=int)
        for k in range(batch_size):
            images[k] = batch_tuple[k]['features']
            labels[k] = batch_tuple[k]['label']
        
        predict = classyfier(images)
        loss = criterion(predict, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_value = loss.detach()
    
    # if (epoch % 10 == 0):
    print('Epoch {} Loss = {}'.format(epoch, loss_value))

Epoch 0 Loss = 1.2448004484176636
Epoch 1 Loss = 1.2351857423782349
Epoch 2 Loss = 1.1250107288360596


In [1474]:
class_size = 600
num_classes = 5
first_class = 95
train_size = class_size * 5
batch_size = 100
batches_in_buff = 10
buff_size = batch_size * batches_in_buff
loader = BatchLoader(class_size, num_classes, first_class, batch_size, batches_in_buff, 'Data/PickledClasses/')

correct = 0
total = 0
for i in range(train_size // batch_size):
    batch_tuple = loader.batch_load(i)
    images = torch.zeros(batch_size, 2048, device=device, requires_grad=False)
    labels = torch.zeros(batch_size, device=device, requires_grad=False, dtype=int)
    for k in range(batch_size):
        images[k] = batch_tuple[k]['features']
        labels[k] = batch_tuple[k]['label'] # don't forget about this
        
    predict = classyfier(images)
    _, predicted = torch.max(predict.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

accuracy = 100 * correct / total
print('Accuracy on FSL task = {} %'.format(accuracy))

Accuracy on FSL task = 74 %


# Saving and loading models

In [1465]:
torch.save(G.to('cpu').state_dict(), 'Models/G')
G.to(device)

DeltaEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=8, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2056, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=2048, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [68]:
torch.save(classyfier.to('cpu').state_dict(), 'Models/classyfier')
classyfier.to(device)

Classyfier(
  (fc): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=5, bias=True)
    (3): Softmax(dim=1)
  )
)

In [4]:
classyfier.load_state_dict(torch.load("Models/classyfier"))

<All keys matched successfully>

In [8]:
G.load_state_dict(torch.load("Models/G"))

<All keys matched successfully>

In [39]:
episode = torch.load('episode.pt')
episode.shape

torch.Size([1, 5, 1, 2048])